# animal_classification **using** Convolutional Neural Network

###importing necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import datetime
import time

###Looking up our image **dataset**

In [2]:
#Default dimensions
img_width, img_height = 224, 224

#Create a bottleneck file
top_model_weights_path = 'bottleneck_fc_model.h5'

# loading up our datasets
train_data_dir = '/workspaces/animal_classification/dataset'
validation_data_dir = '/workspaces/animal_classification/dataset'
test_data_dir = '/workspaces/animal_classification/dataset'

# number of epochs to train top model
epochs = 10

# batch size used by flow_from_directory and predict_generator
batch_size = 50

###Loading vgc16 model

In [ ]:
vgg16 = applications.VGG16(include_top=False, weights='imagenet')

In [4]:
datagen = ImageDataGenerator(rescale=1. / 255)  #needed to create the bottleneck .npy files

### Creation of weights/features with VGG16

In [ ]:
#run this only once
start = datetime.datetime.now()

generator = datagen.flow_from_directory(
train_data_dir,
target_size=(img_width, img_height),
batch_size=batch_size,
class_mode=None,
shuffle=False)

nb_train_samples = len(generator.filenames)
num_classes = len(generator.class_indices)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))

# Use predict instead of predict_generator
bottleneck_features_train = vgg16.predict(generator, steps=predict_size_train) # Changed to predict, added steps argument

np.save('bottleneck_features_train.npy', bottleneck_features_train)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

In [ ]:
#run this only once
start = datetime.datetime.now()
generator = datagen.flow_from_directory(
     validation_data_dir,
     target_size=(img_width, img_height),
     batch_size=batch_size,
     class_mode=None,
     shuffle=False)

nb_validation_samples = len(generator.filenames)

predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

# Use predict instead of predict_generator and add the steps argument
bottleneck_features_validation = vgg16.predict(generator, steps=predict_size_validation)

np.save('bottleneck_features_validation.npy', bottleneck_features_validation)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

In [ ]:
#run this only once
start = datetime.datetime.now()
generator = datagen.flow_from_directory(
     test_data_dir,
     target_size=(img_width, img_height),
     batch_size=batch_size,
     class_mode=None,
     shuffle=False)
nb_test_samples = len(generator.filenames)
predict_size_test = int(math.ceil(nb_test_samples / batch_size))
# Replace predict_generator with predict and add the steps argument
bottleneck_features_test = vgg16.predict(generator, steps=predict_size_test)
np.save('bottleneck_features_test.npy', bottleneck_features_test)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

###Loading training, validation and testing data

In [ ]:
#training data
generator_top = datagen.flow_from_directory(
         train_data_dir,
         target_size=(img_width, img_height),
         batch_size=batch_size,
         class_mode='categorical',
         shuffle=False)
nb_train_samples = len(generator_top.filenames)
num_classes = len(generator_top.class_indices)

# load the bottleneck features saved earlier
train_data = np.load('bottleneck_features_train.npy')

# get the class lebels for the training data, in the original order
train_labels = generator_top.classes

# convert the training labels to categorical vectors
train_labels = to_categorical(train_labels, num_classes=num_classes)

In [ ]:
#validation data
generator_top = datagen.flow_from_directory(
         validation_data_dir,
         target_size=(img_width, img_height),
         batch_size=batch_size,
         class_mode=None,
         shuffle=False)

nb_validation_samples = len(generator_top.filenames)

validation_data = np.load('bottleneck_features_validation.npy')

validation_labels = generator_top.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

In [ ]:
#testing data
generator_top = datagen.flow_from_directory(
         test_data_dir,
         target_size=(img_width, img_height),
         batch_size=batch_size,
         class_mode=None,
         shuffle=False)

nb_test_samples = len(generator_top.filenames)

test_data = np.load('bottleneck_features_test.npy')

test_labels = generator_top.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

##Training of model

In [ ]:
start = datetime.datetime.now()
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3)))
model.add(Dropout(0.5))
model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3)))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4), # Changed lr to learning_rate
              metrics=['acc'])

history = model.fit(train_data, train_labels,
      epochs=10,
      batch_size=batch_size,
      validation_data=(validation_data, validation_labels))

model.save_weights('bottleneck_fc_model.weights.h5')

(eval_loss, eval_accuracy) = model.evaluate(
 validation_data, validation_labels, batch_size=batch_size, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
print("[INFO] Loss: {}".format(eval_loss))
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

In [ ]:
model.summary()

In [ ]:
#Graphing our training and validation
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

###Model Evaluation on Testing Set

In [ ]:
model.evaluate(test_data, test_labels)

###Classification metrics

In [ ]:
print('test data', test_data)
preds = np.round(model.predict(test_data),0)
#to fit them into classification metrics and confusion metrics, some additional modificaitions are required
print('rounded test_labels', preds)

In [ ]:
animals = ['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin', 'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra']
classification_metrics = metrics.classification_report(test_labels, preds, target_names=animals )
print(classification_metrics)

###Confusion Matrix

In [19]:
#Since our data is in dummy format we put the numpy array into a dataframe and call idxmax axis=1 to return the column
#label of the maximum value thos creating a categorical variable
#Basically, flipping a dummy variable back to it's categorical variable
categorical_test_labels = pd.DataFrame(test_labels).idxmax(axis=1)
categorical_preds = pd.DataFrame(preds).idxmax(axis=1)

In [20]:
confusion_matrix= confusion_matrix(categorical_test_labels, categorical_preds)

In [21]:
def plot_confusion_matrix(cm, classes,
             normalize=False,
             title='Confusion matrix',
             cmap=plt.cm.Blues):
#Add Normalization Option
    '''prints pretty confusion metric with normalization option '''
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(confusion_matrix, ['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin', 'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra'])

In [ ]:
#Those numbers are all over the place. Now turning normalize= True
plot_confusion_matrix(confusion_matrix,
                      ['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin', 'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra'],
                     normalize=True)

###Testing images on model

In [24]:
def read_image(file_path):
    print("[INFO] loading and preprocessing image...")
    image = load_img(file_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image /= 255.
    return image

In [25]:
def test_single_image(path):
    animals = ['Bear', 'Bird', 'Cat', 'Cow', 'Deer', 'Dog', 'Dolphin', 'Elephant', 'Giraffe', 'Horse', 'Kangaroo', 'Lion', 'Panda', 'Tiger', 'Zebra']
    images = read_image(path)
    time.sleep(.5)
    bt_prediction = vgg16.predict(images)
    # Use predict instead of predict_proba and extract probabilities
    preds = model.predict(bt_prediction)
    # Assuming your model has a softmax output layer, preds will contain probabilities
    for idx, animal, x in zip(range(0,6), animals , preds[0]):
        print("ID: {}, Label: {} {}%".format(idx, animal, round(x*100,2) ))
    print('Final Decision:')
    time.sleep(.5)
    for x in range(3):
        print('.'*(x+1))
        time.sleep(.2)
    # Use predict and argmax to get predicted class
    class_predicted = np.argmax(model.predict(bt_prediction), axis=-1)
    class_dictionary = generator_top.class_indices
    inv_map = {v: k for k, v in class_dictionary.items()}
    print("ID: {}, Label: {}".format(class_predicted[0], inv_map[class_predicted[0]]))
    return load_img(path)

In [26]:
#path of image that is used for testing
Beer = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Bear/Bear_10.jpg'

In [ ]:
test_single_image(Beer)

In [28]:
#path of image that is used for testing
Bird = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Bird/Bird_10_3.jpg'

In [ ]:
test_single_image(Bird)

In [30]:
#path of image that is used for testing
Cat = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Cat/Cat_14_2.jpg'

In [ ]:
test_single_image(Cat)

In [32]:
#path of image that is used for testing
Cow = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Cow/Cow_19_1.jpg'

In [ ]:
test_single_image(Cow)

In [34]:
#path of image that is used for testing
Deer = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Deer/Deer_13.jpg'

In [ ]:
test_single_image(Deer)

In [36]:
#path of image that is used for testing
Dog = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Dog/Dog_13_3.jpg'

In [ ]:
test_single_image(Dog)

In [38]:
#path of image that is used for testing
Dolphin = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Dolphin/Dolphin_10.jpg'

In [ ]:
test_single_image(Dolphin)

In [40]:
#path of image that is used for testing
Elephant = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Elephant/Elephant_12_1.jpg'

In [ ]:
test_single_image(Elephant)

In [42]:
#path of image that is used for testing
Giraffe = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Giraffe/Giraffe_14_3.jpg'

In [ ]:
test_single_image(Giraffe)

In [44]:
#path of image that is used for testing
Horse = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Horse/Horse_24_3.jpg'

In [ ]:
test_single_image(Horse)

In [46]:
#path of image that is used for testing
Kangaroo = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Kangaroo/Kangaroo_23.jpeg'

In [ ]:
test_single_image(Kangaroo)

In [48]:
#path of image that is used for testing
Lion = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Lion/Lion_2_4.jpg'

In [ ]:
test_single_image(Lion)

In [50]:
#path of image that is used for testing
Panda = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Panda/Panda_27.jpeg'

In [ ]:
test_single_image(Panda)

In [52]:
  #path of image that is used for testing
Tiger = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Tiger/Tiger_22_3.jpg'

In [ ]:
test_single_image(Tiger)

In [54]:
#path of image that is used for testing
Zebra = '/content/drive/MyDrive/Projects-20240722T093004Z-001/Projects/animal_classification/Animal Classification/dataset/Zebra/Zebra_9_4.jpg'

In [ ]:
test_single_image(Zebra)